# Tableau and the File System

The Langchain community provides out of the box tools that can make the Superstore Agent more powerful. Now that this agent is able to query Tableau data sources, we can give it tools to interact with the file system so it accomplish the following:

1. Write data outputs to the file system
2. Write insights to the file system
3. List outputs
4. Read outputs back to you
   
This is possible thanks to the [filesystem toolkit](https://python.langchain.com/docs/integrations/tools/filesystem/).

![Tableau dashboards inside bubbles](./assets/embed_samples.png)

Start by importing the necessary packages:

In [2]:
import os

from langchain_community.agent_toolkits import FileManagementToolkit

## Initialize the File System Toolkit

The next step is to create a temporary working directory and initialize the File System toolkit. That way we can get then tools we need for this exercise such as `read_tool`, `write_tool` and `list_tool`.

When combined with data from Tableau the Superstore Agent will be capable of several interesting scenarios.

In [3]:
# Use "temp" folder in the current Jupyter Notebook directory
temp_dir = "temp"

# Create the temp directory if it doesn't exist
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

#  initialize the desired toolkit
toolkit = FileManagementToolkit(
    root_dir=temp_dir,
    selected_tools=["read_file", "write_file", "list_directory"]
)

# store all of the tools in the toolkit as 'tools'
file_tools = toolkit.get_tools()

# unpack the tools we will use in the exercise
read_tool, write_tool, list_tool, *_ = file_tools

## Testing the Filesystem Toolkit

To get acquainted with these new tools, let's "invoke" them individually without an agent to inspect their output.

Let's first start with writing a new file, you can inspect the physical output yourself inside of the `experimental/notebooks/temp` folder:

In [41]:
# Create the file before writing to it
file_path = os.path.join(temp_dir, "example.txt")
with open(file_path, "w") as f:
    pass  # Create an empty file

write_tool.invoke({"file_path": "hello_superstore.txt", "text": "Hello Superstore!"})

'File written successfully to hello_superstore.txt.'

Now list the files inside of the `experimental/notebooks/temp` folder with the `list_tool`:

In [43]:
# List files in the working directory using `list_tool`
files = list_tool.invoke({})

# Split the raw output into a list of filenames, removing empty entries
file_list = [name for name in files.strip().split('\n') if name]

# Format the list into Markdown list items
markdown_list_items = []
if file_list:
    for file_name in sorted(file_list): # Sort alphabetically for consistency
        markdown_list_items.append(f"* {file_name}") # Prepend Markdown list marker '* '
        # You could use "- " instead: markdown_list_items.append(f"- {file_name}")
else:
    markdown_list_items.append("* (Directory is empty)") # Indicate if empty

# Join the Markdown list items back into a single string
markdown_output = "\n".join(markdown_list_items)

# Print the final Markdown formatted string
print("\nFormatted Markdown List Output:")
print(markdown_output)


Formatted Markdown List Output:
* .gitignore
* .gitkeep
* hello_superstore.txt


NOTE: `.gitkeep` & `.gitignore` were already present inside the "temp" folder to make it a temporary placeholder

Lets inspect the contents of the `hello_superstore.txt` file with the `read_tool`:

In [44]:
# Invoke the read_tool, providing the relative path within the root_dir
file_content = read_tool.invoke({"file_path": "hello_superstore.txt"})

# The result is the content of the file as a string
print("\n--- Successfully Read File Content ---")
print(file_content)
print("--- End of File Content ---")


--- Successfully Read File Content ---
Hello Superstore!
--- End of File Content ---


## Agent Superstore becomes a Writer!

Now that we are familiar with these new tools, let's equip our agent so it can use them in combination with the Tableau Data Q&A tool we learned about in the [previous lession](./02_tableau_datasource_qa.ipynb). That way the agent can read and write files using factual information from Superstore's operations.

Agents can be relatively powerless without data since it allows them make decisions and produce useful outputs. Otherwise, we would be stuck having agent who write jokes or poems but this doesn't help Superstore be a more efficient company at all.

### Setup the Data Source Q&A Tool

Import dependencies and environment variables used by the agent and the Data Source Q&A tool:

In [5]:
# to access environment variables
import os
from dotenv import load_dotenv

# for displaying pretty results in the notebook
from IPython.display import display, Markdown

# Langgraph packages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage # This will help with the markdown section

# langchain_tableau packages
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa


# loads environment variables into Python script
load_dotenv()

# initialize variables with these secure values
tableau_server = os.getenv('TABLEAU_DOMAIN')
tableau_site = os.getenv('TABLEAU_SITE')
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID')
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID')
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET')
tableau_api_version = os.getenv('TABLEAU_API_VERSION')
tableau_user = os.getenv('TABLEAU_USER')
datasource_luid = os.getenv('DATASOURCE_LUID')
tooling_model = os.getenv('TOOLING_MODEL')

# Initalize the tool for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_model
)

# add the data query tool and the file system tools to a List to give to the agent
tools = [ analyze_datasource, read_tool, write_tool, list_tool ]

### Run & Format our Chats

We will likewise add the running function we've been using so far to improve readability and handle any errors.

You may be trying this out on your own environment, so some good error handling never hurt anybody:

In [6]:
def run_agent_and_display_markdown(agent, user_prompt):
    """Invokes the agent and displays the final answer using Markdown."""
    print(f"Running agent with prompt: '{user_prompt}'")
    messages = {"messages": [HumanMessage(content=user_prompt)]} # Use HumanMessage directly

    try:
        # Invoke the agent to get the final result
        # The result dictionary structure might vary slightly depending on the LangGraph version
        # but typically contains 'messages' with the conversation history.
        result = agent.invoke(messages)

        # Extract the final message (usually the last AIMessage)
        if result and 'messages' in result and result['messages']:
            final_message = result['messages'][-1]
            # Check if it's an AI message and has content
            if hasattr(final_message, 'content'):
                final_answer = final_message.content
                print("\n--- Agent Final Answer ---")
                display(Markdown(final_answer))
                print("--------------------------\n")
            else:
                print("Could not extract content from the final message.")
                print("Final message:", final_message)
        else:
            print("Agent did not return the expected result structure.")
            print("Result:", result)

    except Exception as e:
        print(f"An error occurred while running the agent: {e}")
        # Potentially display the error or log it
        display(Markdown(f"**Error:**\n```\n{e}\n```"))

### Setup the Superstore Agent

We use the same prompt as the [previous lession](./02_tableau_datasource_qa.ipynb) but we include new instructions to help the Agent understand its new tools and how they open new use cases:

In [7]:
# Agent Identity Definition
identity = """
You are **Agent Superstore**, the veteran AI analyst who has spent years exploring the aisles of the legendary Superstore dataset.
A dataset many Tableau users know and love! 
You live and breathe Superstore data: sales, profits, regions, categories, customer segments, shipping modes, you name it.

Your special mission **today at Tableau Conference 2025** is to help attendees experience the power of Tableau for Langchain Agents. 
You'll be their guide, using this new tool to query the Superstore dataset directly and uncover insights in real-time.

**When you first introduce yourself:**
1.  Greet the attendees and welcome them to the Tableau Conference 2025 hands-on session.
2.  Introduce yourself as Agent Superstore, the AI expert on the classic Superstore dataset.
3.  Briefly explain your purpose: to demonstrate Tableau analytics via agents
"""

# Main System Prompt
system_prompt = f"""**Agent Identity:**
{identity}

**Core Instructions:**

You are an AI Analyst specifically designed to generate data-driven insights from datasets using the tools provided. 
Your goal is to provide answers, guidance, and analysis based on the data accessed via your tools. 
Remember your audience: Tableau users at a conference session, likely familiar with Superstore aka the best dataset ever created.

**Tool Usage Strategy:**

You have access to the following tool:

1.  **`tableau_query_tool` (Data Source Query):** This is your primary tool for interacting with data.
    * **Prioritize this tool** for nearly all user requests asking for specific data points, aggregations, comparisons, trends, or filtered information from datasets.
    * Use it to find specific values (e.g., sales for 'Technology' in 'West' region), calculate aggregates (e.g., `SUM(Sales)`, `AVG(Profit Ratio)`), filter data (e.g., orders in 2023), group data (e.g., sales `BY Category`), and find rankings (e.g., top 5 products by quantity).
    * Be precise in formulating the queries based on the user's request.

**Response Guidelines:**

* **Grounding:** Base ALL your answers strictly on the information retrieved from your available tools.
* **Clarity:** Always answer the user's core question directly first.
* **Source Attribution:** Clearly state that the information comes from the **dataset** accessed via the Tableau tool (e.g., "According to the data...", "Querying the datasource reveals...").
* **Structure:** Present findings clearly. Use lists or summaries for complex results like rankings or multiple data points. Think like a mini-report derived *directly* from the data query.
* **Tone:** Maintain a helpful, and knowledgeable, befitting your Tableau Superstore expert persona.

**Crucial Restrictions:**
* **DO NOT HALLUCINATE:** Never invent data, categories, regions, or metrics that are not present in the output of your tools. If the tool doesn't provide the answer, state that the information isn't available in the queried data.
"""

# initialize a Large Language Model to be the "brains" of the Agent
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

superstore_agent = create_react_agent(model=model, tools=tools, prompt=system_prompt)

## Let's Rock & Roll!

